In [1]:
%matplotlib inline
import time
import numpy as np
import xesmf as xe
import xarray as xr
import netCDF4 as nc
import cmocean as cm
import cartopy.crs as ccrs
import matplotlib.ticker
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import matplotlib.gridspec as gridspec
import warnings
warnings.filterwarnings('ignore')

In [2]:
g = 9.8
rho0 = 1035.0
m2_period = 236 # in hr (19 cycles)
radius = 6378e+3

In [3]:
tpxo = xr.open_dataset('/g/data/nm03/TPXO9v5a/h_tpxo9.v5a.nc')
tpgd = xr.open_dataset('/g/data/nm03/TPXO9v5a/gridtpxo9v5a.nc')
pha_tpxo = (tpxo.hp.isel({'nc':0}).transpose()-180) /180 *np.pi
amp_tpxo = tpxo.ha.isel({'nc':0}).transpose()
lon_z = np.array(tpgd.lon_z.transpose())/180 *np.pi
lat_z = np.array(tpgd.lat_z.transpose())/180 *np.pi
lon_u = np.array(tpgd.lon_u.transpose())/180 *np.pi
lat_u = np.array(tpgd.lat_u.transpose())/180 *np.pi
lon_v = np.array(tpgd.lon_v.transpose())/180 *np.pi
lat_v = np.array(tpgd.lat_v.transpose())/180 *np.pi
hz_tpxo = np.array(tpgd.hz.transpose())
hu_tpxo = np.array(tpgd.hu.transpose())
hv_tpxo = np.array(tpgd.hv.transpose())
ny, nx  = np.array(tpgd.ny), np.array(tpgd.nx)

### Construct TPXO elevation $\eta$

In [4]:
t_12cyc = np.arange(m2_period)
nt_tpxo = t_12cyc.size
omega_m2 = np.full((nt_tpxo,ny.size,nx.size),(2*np.pi/(12.4206014)))
elev_tpxo = np.array(amp_tpxo)[None,:,:] * np.cos(omega_m2 * t_12cyc[:,None,None] - np.array(pha_tpxo)[None,:,:])

### Construct $\eta_{eq}$ and $\eta_{sal}$

In [5]:
love = 0.693
eta_eq = love * 0.242334 * np.cos(np.pi/180*lat_z[None,:,:])**2 * np.exp(-1j*(omega_m2 * t_12cyc[:,None,None]+2*np.pi/180*lon_z[None,:,:]))

In [6]:
SAL = xr.open_dataset('/g/data/nm03/OTPSnc/DATA/load_file.nc')
pha_SAL = (SAL.hp.isel({'nc':0}).transpose()-180) /180 *np.pi
amp_SAL = SAL.ha.isel({'nc':0}).transpose()
eta_sal = np.array(amp_SAL)[None,:,:] * np.cos(omega_m2 * t_12cyc[:,None,None] - np.array(pha_SAL)[None,:,:])

In [7]:
eta = eta_sal + eta_eq

In [8]:
eta.shape

(236, 1081, 2160)

In [9]:
tpxo_vel = xr.open_dataset('/g/data/nm03/TPXO9v5a/u_tpxo9.v5a.nc')
ua_tpxo = np.array(tpxo_vel.ua.isel({'nc':0}).transpose()/100)
va_tpxo = np.array(tpxo_vel.va.isel({'nc':0}).transpose()/100)
up_tpxo = np.array((tpxo_vel.up.isel({'nc':0}).transpose() - 180) /180 *np.pi +13*np.pi/16)
vp_tpxo = np.array((tpxo_vel.vp.isel({'nc':0}).transpose() - 180) /180 *np.pi +13*np.pi/16)

In [10]:
u_tpxo = ua_tpxo[None,:,:] * np.cos(omega_m2 * t_12cyc[:,None,None] - up_tpxo[None,:,:]) * hu_tpxo[None,:,:]
v_tpxo = va_tpxo[None,:,:] * np.cos(omega_m2 * t_12cyc[:,None,None] - vp_tpxo[None,:,:]) * hv_tpxo[None,:,:]

### Interp eta to u and v points

In [11]:
ds_elev_z = xr.Dataset(data_vars={"elev": (('nt','ny','nx'), elev_tpxo),
                                 },
                       coords={"lon": (('ny', 'nx'), lon_z), 
                               "lat": (('ny', 'nx'), lat_z)})
ds_grid_u = xr.Dataset({"lat": (["ny","nx"], lat_u),
                        "lon": (["ny","nx"], lon_u)})
ds_grid_v = xr.Dataset({"lat": (["ny","nx"], lat_v),
                        "lon": (["ny","nx"], lon_v)})

regridder_z2u = xe.Regridder(ds_elev_z, ds_grid_u, "bilinear", extrap_method="inverse_dist")
regridder_z2v = xe.Regridder(ds_elev_z, ds_grid_v, "bilinear", extrap_method="inverse_dist")

ds_elev_u = regridder_z2u(ds_elev_z)
ds_elev_v = regridder_z2v(ds_elev_z)

In [12]:
elev_u = np.array(ds_elev_u["elev"])
elev_v = np.array(ds_elev_v["elev"])
print(elev_u.shape)
print(elev_v.shape)

(236, 1081, 2160)
(236, 1081, 2160)


In [13]:
# interp from v to z points
divP1 = 1/(radius*np.cos(lat_v[None,1:-1,:])) * \
        (v_tpxo[:,2:,:]*elev_v[:,2:,:]*np.cos(lat_v[None,2:,:]) - v_tpxo[:,:-2,:]*elev_v[:,:-2,:]*np.cos(lat_v[None,:-2,:])) / (lat_v[None,2:,:] - lat_v[None,:-2,:])
divP2 = 1/(radius*np.cos(lat_u[None,:,1:-1])) * (u_tpxo[:,:,2:]*elev_u[:,:,2:] - u_tpxo[:,:,:-2]*elev_u[:,:,:-2]) / (lon_u[None,:,2:] - lon_u[None,:,:-2])       

In [14]:
ds_divP_v = xr.Dataset(data_vars={"divP1": (('nt','ny','nx'), divP1),
                                 },
                       coords={"lon": (('ny', 'nx'), lon_u[1:-1,:]), 
                               "lat": (('ny', 'nx'), lat_u[1:-1,:])})

ds_divP_u = xr.Dataset(data_vars={"divP2": (('nt','ny','nx'), divP2),
                                 },
                       coords={"lon": (('ny', 'nx'), lon_u[:,1:-1]), 
                               "lat": (('ny', 'nx'), lat_u[:,1:-1])})

ds_grid_z = xr.Dataset({"lat": (["ny","nx"], lat_z),
                        "lon": (["ny","nx"], lon_z)})

regridder_div_v2z = xe.Regridder(ds_divP_v, ds_grid_z, "bilinear", extrap_method="inverse_dist")
regridder_div_u2z = xe.Regridder(ds_divP_u, ds_grid_z, "bilinear", extrap_method="inverse_dist")

ds_divP1_z = regridder_div_v2z(ds_divP_v)
ds_divP2_z = regridder_div_u2z(ds_divP_u)

In [15]:
P = g * rho0 * np.nanmean(np.array(ds_divP1_z["divP1"] + ds_divP2_z["divP2"]), axis=0) 
print(P.shape)

(1081, 2160)


In [16]:
grad_the = 1/radius * (eta[:,2:,:]-eta[:,:-2,:]) / (lat_z[None,2:,:]-lat_z[None,:-2,:])
grad_phi = 1/(radius * np.cos(lat_z[None,:,1:-1])) * (eta[:,:,2:]-eta[:,:,:-2]) / (lon_z[None,:,2:]-lon_z[None,:,:-2])

In [17]:
ds_grad_zlat = xr.Dataset(data_vars={"grad_the": (('nt','ny','nx'), grad_the),
                                    },
                          coords={"lon": (('ny', 'nx'), lon_z[1:-1,:]), 
                                  "lat": (('ny', 'nx'), lat_z[1:-1,:])})

ds_grad_zlon = xr.Dataset(data_vars={"grad_phi": (('nt','ny','nx'), grad_phi),
                                    },
                          coords={"lon": (('ny', 'nx'), lon_z[:,1:-1]), 
                                  "lat": (('ny', 'nx'), lat_z[:,1:-1])})

ds_grid_u = xr.Dataset({"lat": (["ny","nx"], lat_u),
                        "lon": (["ny","nx"], lon_u)})

ds_grid_v = xr.Dataset({"lat": (["ny","nx"], lat_v),
                        "lon": (["ny","nx"], lon_v)})

regridder_grad_z2v = xe.Regridder(ds_grad_zlat, ds_grid_v, "bilinear", extrap_method="inverse_dist")
regridder_grad_z2u = xe.Regridder(ds_grad_zlon, ds_grid_u, "bilinear", extrap_method="inverse_dist")

ds_grad_v = regridder_grad_z2v(ds_grad_zlat)
ds_grad_u = regridder_grad_z2u(ds_grad_zlon)

grad_v = np.array(ds_grad_v["grad_the"])
grad_u = np.array(ds_grad_u["grad_phi"])

In [18]:
ds_grad_v = xr.Dataset(data_vars={"gradW1": (('nt','ny','nx'), -np.array(v_tpxo)*grad_v),
                                 },
                       coords={"lon": (('ny', 'nx'), lon_v), 
                               "lat": (('ny', 'nx'), lat_v)})

ds_grad_u = xr.Dataset(data_vars={"gradW2": (('nt','ny','nx'), np.array(u_tpxo)*grad_u),
                                 },
                       coords={"lon": (('ny', 'nx'), lon_u), 
                               "lat": (('ny', 'nx'), lat_u)})

ds_grid_z = xr.Dataset({"lat": (["ny","nx"], lat_z),
                        "lon": (["ny","nx"], lon_z)})

regridder_grad_v2z = xe.Regridder(ds_grad_v, ds_grid_z, "bilinear", extrap_method="inverse_dist")
regridder_grad_u2z = xe.Regridder(ds_grad_u, ds_grid_z, "bilinear", extrap_method="inverse_dist")

ds_gradW1_z = regridder_grad_v2z(ds_grad_v)
ds_gradW2_z = regridder_grad_u2z(ds_grad_u)

W1 = np.array(ds_gradW1_z["gradW1"])
W2 = np.array(ds_gradW2_z["gradW2"])

In [19]:
W = g * rho0 * np.nanmean(W1 + W2, axis=0)

In [20]:
D = W - P

In [21]:
D.shape

(1081, 2160)

In [22]:
ds_dissip = xr.Dataset(data_vars={"TPXO_dissip_real": (('ny','nx'), D.real),
                                  "TPXO_dissip_mag": (('ny','nx'), np.abs(D)),
                                 },
                       coords={"lon": (('ny', 'nx'), lon_z), 
                               "lat": (('ny', 'nx'), lat_z)})
ds_dissip.to_netcdf('/g/data/nm03/lxy581/tpxo9v5a/TPXO_M2_dissip.nc')

In [23]:
# dissip = xr.open_dataset('/g/data/nm03/lxy581/tpxo9v5a/TPXO_M2_dissip.nc')

In [24]:
# np.nanmean(np.array(dissip["TPXO_dissip_mag"]))

In [25]:
# dissip["TPXO_dissip_mag"].plot(x='lon',y='lat')

In [26]:
# ds_dissip = xr.Dataset(data_vars={"TPXO_dissip": (('ny','nx'), np.array(dissip["TPXO_dissip_mag"])),
#                                  },
#                        coords={"lon": (('ny', 'nx'), lon_z), 
#                                "lat": (('ny', 'nx'), lat_z)})

### Convert D to MOM6 grid

In [27]:
file_stat = '/g/data/nm03/lxy581/archive/tides_025_SAL_JSL_masked/output003/ocean_static.nc'
stat  = xr.open_dataset(file_stat)
yh, xh = stat.yh, stat.xh
depth  = stat['deptho']
geolat = stat['geolat']
geolon = stat['geolon']
geolon_u = stat['geolon_u']
geolat_u = stat['geolat_u']
geolon_v = stat['geolon_v']
geolat_v = stat['geolat_v']
area   = stat['areacello']
fac_dep = xr.where(depth > 1000, 1, np.nan) 
fac_lat = xr.where((geolat < 75) & (geolat > -75), 1, np.nan)
fac = np.array(fac_dep) * np.array(fac_lat)

In [28]:
topog = xr.open_dataset('/home/581/lxy581/tidal_param/MOM6-examples/ocean_only/tides_025/INPUT/ocean_topog.nc')
depth = topog.depth
depth_z = np.array(depth)
depth_z[depth_z==0]=np.nan
depth_da = xr.Dataset(data_vars={'depth_xr': (('yh','xh'), depth_z),
                                }, 
                      coords={'lon': (('yh', 'xh'), np.array(geolon)),
                              'lat': (('yh', 'xh'), np.array(geolat))})
land = xr.where(np.isnan(depth_da.depth_xr[:,:].rename('land')), 1, np.nan)

In [ ]:
ds_mom6_grid = xr.Dataset({"lat": (["yh","xh"], np.array(geolat)),
                           "lon": (["yh","xh"], np.array(geolon))})
print('interpolating...')
regridder_dissip = xe.Regridder(ds_dissip, ds_mom6_grid, "bilinear", extrap_method="inverse_dist")
print('regriddering...')
ds_D_mom6 = regridder_dissip(ds_dissip)
print('getting interpolated dissipation...')
TPXO_dissip = np.array(ds_D_mom6["TPXO_dissip_mag"])
print('calculating global sum...')
global_dissip  = np.nansum(TPXO_dissip * np.array(area)) / 1e+12

interpolating...


In [ ]:
deep_dissip = np.nansum(TPXO_dissip * np.array(area)*fac_dep) / 1e+12
print('Global M2 tidal energy dissipation: %.1fTW'%global_dissip)
print('Open ocean dissipation fraction: %d%%'%(deep_dissip/global_dissip*100))

In [ ]:
varlist = ['TPXO_dissip']
title   = ['TPXO M2 tidal energy dissipation \n Global sum: %.2f TW'%(global_dissip)]
tick_locs = np.array([1e-4,1e-3,1e-2,1e-1,1e-0])
tick_labels = np.array(["10$^{-4}$","10$^{-3}$","10$^{-2}$","10$^{-1}$","10$^{0}$"])
# tick_locs = np.array([-0.01,-0.005,0,0.005,0.01])
# tick_labels = np.array(["-0.01","-.005","0","0.005","0.01"])

fig = plt.figure(figsize=(22, 10))
axs = []

gridsubs = gridspec.GridSpec(1,1)
for gs in gridsubs:
    axs.append(plt.subplot(gs))

for I, (ax, var) in enumerate(zip(axs, varlist)):

    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['bottom'].set_visible(False)
    ax.spines['left'].set_visible(False)
    ax.get_xaxis().set_ticks([])
    ax.get_yaxis().set_ticks([])
    
    ax = plt.axes(projection=ccrs.Robinson(central_longitude=-100))
    # Add model land mask
    land.plot.contourf(ax=ax, x='lon', y='lat', colors='darkgrey', zorder=2, transform=ccrs.PlateCarree(), add_colorbar=False)
    # Add model coastline
    land.fillna(0).plot.contour(ax=ax, x='lon', y='lat', colors='k', levels=[0, 1], transform=ccrs.PlateCarree(), add_colorbar=False, linewidths=0.5)

    # p1 = ax.pcolormesh(geolon, geolat, globals()[var], transform=ccrs.PlateCarree(), cmap=cm.cm.balance, vmin=-0.01, vmax=0.01)
    p1 = ax.pcolormesh(geolon, geolat, globals()[var], transform=ccrs.PlateCarree(), cmap=matplotlib., norm=LogNorm(vmin=1e-4, vmax=1e+0))
    plt.title(title[I],fontsize=24)
  
    if I==0:
        ax_cb = plt.axes([0.86, 0.13, 0.01, 0.68])
    
    cb1 = plt.colorbar(p1, cax=ax_cb, orientation='vertical', extend='both')
    cb1.ax.set_ylabel('Energy dissipation (W m$^{-2}$)',fontsize=20,rotation=270,labelpad=25);
    cb1.ax.tick_params(labelsize=16)
    cb1.locator   = matplotlib.ticker.FixedLocator(tick_locs)
    cb1.formatter = matplotlib.ticker.FixedFormatter(tick_labels)

    if I==0:
        ax.set_position([0.1,0.1,0.75,0.75])
        
plt.savefig('/g/data/nm03/lxy581/exps_figs/TPXO_M2_dissip_log.png', dpi=300, bbox_inches='tight')